In [1]:
import json
import pandas as pd
import requests

In [2]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [3]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [17]:
with open("../shared-folder-gald/keys2.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [18]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [19]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [20]:
def get_repost_info(username, access_token,fields ="id, username, favorites_count" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/reposted_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [21]:
access_token = get_access_token(client_key, client_secret)

keeping track with calls
27.02 - we ran keys2 unril 1000 

1.03 - keys2 1001:2002

1.03 - keys1 2002:3003
2.03 - keys1 3003:4004
2.03 - keys2 4004:5005


In [22]:
reposted_videos = []
for username in usernames[4004:5005]:
    reposted_videos.append(get_repost_info(username, access_token))

In [23]:
reposted_videos

[{'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 14197,
    'id': 7446971375669120302,
    'username': 'atl5d'},
   {'username': 'blkgrlblkgrl2',
    'favorites_count': 5711,
    'id': 7475518519052864814},
   {'id': 7447270232839851310,
    'username': 'thebeepartner',
    'favorites_count': 3293},
   {'favorites_count': 636,
    'id': 7471479084908924202,
    'username': 'pharmunique_'},
   {'favorites_count': 13220,
    'id': 7468148476640120095,
    'username': 'jordantalexander'},
   {'favorites_count': 16743,
    'id': 7470744152666295598,
    'username': 'abbywinters80'},
   {'favorites_count': 25037,
    'id': 7434215807858642218,
    'username': 'levertthebassman'},
   {'favorites_count': 7266,
    'id': 7463976305223044395,
    'username': 'toureshow'},
   {'id': 7413550575503166766,
    'username': 'alexisnikole',
    'favorites_count': 8384},
   {'favorites_count': 10981, 'id': 7473284209365519662, 'username': 'nfbci'},
   {'favorites_count': 73

In [24]:
#saving global reposts for weights in the network
import csv
with open('reposted_videos_raw', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(reposted_videos)

In [25]:
combined = {}
for i, username in enumerate(usernames[4004:5005]):
    combined[username] = reposted_videos[i]
    
combined 


{'projectgrape': {'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 14197,
    'id': 7446971375669120302,
    'username': 'atl5d'},
   {'username': 'blkgrlblkgrl2',
    'favorites_count': 5711,
    'id': 7475518519052864814},
   {'id': 7447270232839851310,
    'username': 'thebeepartner',
    'favorites_count': 3293},
   {'favorites_count': 636,
    'id': 7471479084908924202,
    'username': 'pharmunique_'},
   {'favorites_count': 13220,
    'id': 7468148476640120095,
    'username': 'jordantalexander'},
   {'favorites_count': 16743,
    'id': 7470744152666295598,
    'username': 'abbywinters80'},
   {'favorites_count': 25037,
    'id': 7434215807858642218,
    'username': 'levertthebassman'},
   {'favorites_count': 7266,
    'id': 7463976305223044395,
    'username': 'toureshow'},
   {'id': 7413550575503166766,
    'username': 'alexisnikole',
    'favorites_count': 8384},
   {'favorites_count': 10981, 'id': 7473284209365519662, 'username': 'nfbci'},
   {'favo

In [26]:
rows = []

for key, value in combined.items():
    if value and 'reposted_videos' in value:  # Ensure key has data
        for video in value['reposted_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'video_id': video['id'],  # Video ID
                'creator': video['username'], # Video username
                'favourites': video["favorites_count"]
            })

# Create DataFrame
reposted_df = pd.DataFrame(rows)

# Display DataFrame
print(reposted_df)

              username             video_id           creator  favourites
0         projectgrape  7446971375669120302             atl5d       14197
1         projectgrape  7475518519052864814     blkgrlblkgrl2        5711
2         projectgrape  7447270232839851310     thebeepartner        3293
3         projectgrape  7471479084908924202      pharmunique_         636
4         projectgrape  7468148476640120095  jordantalexander       13220
...                ...                  ...               ...         ...
11111  saltwater_woman  7472867424463867144   ausconservation        1222
11112  saltwater_woman  7471096584160283911  x.arayliashanae_           1
11113  saltwater_woman  7470804633628331285          _manmiya        1661
11114  saltwater_woman  7469744521819999506   treseaireneryan           2
11115  saltwater_woman  7474267783421873413         mooselegs         273

[11116 rows x 4 columns]


In [27]:
reposted_df.to_csv("reposted_videos.csv", mode="a", index=False, header=False)